# Transformer Coded from Scratch

In this Jupyter Notebook, I show my code and some of the notes I used when trying to code a transformer model from scratch. This was an exercise to really understand every part of a transformer architecture and how they interact. I found learning about positional encoding very interesting and the exercise really solidified my understanding of how attention heads compute attention in parallel and combine information at the end for the sake of efficiency. 

The code was quite buggy, naming conventions are misused and I replaced some functions while debugging (commenting out or deleting the origional functions). However, I learnt not to do this as many issues in the code were not to do with the function where the error was but occurred before that and were only flagged later. I enjoyed fixing the code and reflecting on where I went wrong in this process and will use the lessons I have learnt, mainly consistently assigning variables and properly naming objects (sloppy mistakes I know). 

## So take this code with a pinch of salt. It is not something I would present to someone else, I have much higher standards. Take it as proof that an exercise has been completed and learning has been done. 

What are the things i need:

TRansformer strucutre: 

Masking for source and target

embedding layer, positional embedding, mask and then funnel it through the encoder?

decoder

final layer you either softmax of the vocab to find the next item in the sequence or you put it through some more layers

Masking
it needs a matrix of the number of sequences x the longest equence length
it needs a way to change all the padding tokens to 0 and it needs a boolen method to turn that into true and false



class Mak


In [19]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.utils.data as data
import math
import copy

In [20]:
class PositionEncoding(nn.Module):

    def __init__(self, d_model, max_sequence_len):
        super(PositionEncoding, self).__init__()

        #matrix of positional encodings
        pe = torch.zeros(max_sequence_len, d_model)
        position = torch.arange(0, max_sequence_len, dtype=torch.float).unsqueeze(1)
        div_term = torch.exp(torch.arange(0, d_model, 2).float() * (-math.log(10000.0)/d_model))

        pe[:, 0::2] = torch.sin(position * div_term)
        pe[:, 1::2] = torch.cos(position * div_term)

        pe = pe.unsqueeze(0)
        #unsqueeze adds a dimension to make it suitable as a module - extra layer is for batching although
        #that is not happening here
        self.register_buffer('pe', pe)    
    #model buffers are not considered part of the model paramteres and are not updates in training

    def forward(self, x):
        #add pos encodings to te input
        return x + self.pe[:, :x.size(1)]

In [21]:

import torch.nn as nn
import math

class PositionWiseSelfAttention(nn.Module):

    #with attention what do i need to do - split heads, calculate attetion and multiply it to v? 
    #then combine heads again and output? isn't there a linear transformation at the end

    def __init__(self, d_model, num_heads):
        super(PositionWiseSelfAttention, self).__init__()


        assert d_model % num_heads == 0, 'd_model must be divisible by num_heads'

        self.d_model = d_model
        self.num_heads = num_heads
        self.d_k = d_model // num_heads

        self.W_q = nn.Linear(d_model, d_model)
        self.W_k = nn.Linear(d_model, d_model)
        self.W_v = nn.Linear(d_model, d_model)
        self.W_o = nn.Linear(d_model, d_model)

        



    def split_heads(self, x):
        batch_size, seq_len, d_model = x.size()
        return x.view(batch_size, seq_len, self.num_heads, self.d_k).transpose(1, 2)
        


   

    def scaled_dot_product(self, Q, K, V, mask=None):
        attn_scores =  torch.matmul(Q, K.transpose(-2, -1)/math.sqrt(self.d_k))
        if mask is not None:
            attn_scores = attn_scores.masked_fill(mask == 0, -1e9)

        attn_probs = torch.softmax(attn_scores, dim=-1)

        # Multiply by values to obtain the final output
        output = torch.matmul(attn_probs, V)
        return output
    
    def combine_heads(self, x):
        batch_size, _, seq_len, d_k = x.size()
        return x.transpose(1, 2).contiguous().view(batch_size, seq_len, self.d_model)

    def forward(self, Q, K, V, mask=None):

        Q = self.split_heads(self.W_q(Q))
        K = self.split_heads(self.W_k(K))
        V = self.split_heads(self.W_v(V))

        att_output = self.scaled_dot_product(Q, K, V, mask)

        output = self.W_o(self.combine_heads(att_output))
        return output



In [22]:
class PositionWiseFeedForward(nn.Module):

    def __init__(self, d_model, d_big):
        super(PositionWiseFeedForward, self).__init__()

        self.linear_1 = nn.Linear(d_model, d_big)
        self.ReLu = nn.ReLU()
        self.linear_2 = nn.Linear(d_big, d_model)


    def forward(self, x):
        return self.linear_2(self.ReLu(self.linear_1(x)))
    


In [23]:

'''class Encoder(nn.Module):

    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(Encoder, self).__init__()
        self.multi_head_attn = PositionWiseSelfAttention(d_model, num_heads)
        self.ff = PositionWiseFeedForward(d_model, d_big=d_ff)
        self.norm_1 = nn.LayerNorm(d_model)
        self.norm_2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

        

    def forward(self, x, mask):
        step_one = self.multi_head_attn(x, x, x, mask)
        step_two = self.norm_1(x +self.dropout(step_one))
        step_3 = self.ff(step_two)
        step_4 = self.norm_2(step_one + self.dropout(step_3))

        return step_4'''
    

class EncoderLayer(nn.Module):
    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(EncoderLayer, self).__init__()
        self.self_attn = PositionWiseSelfAttention(d_model, num_heads)
        self.feed_forward = PositionWiseFeedForward(d_model, d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.dropout = nn.Dropout(dropout)

    def forward(self, x, mask):
        attn_output = self.self_attn(x, x, x, mask)
        x = self.norm1(x + self.dropout(attn_output))
        
        ff_output = self.feed_forward(x)
        x = self.norm2(x + self.dropout(ff_output))
        return x    


In [24]:
class Decoder(nn.Module):

    def __init__(self, d_model, num_heads, d_ff, dropout):
        super(Decoder, self).__init__()

        self.self_attn = PositionWiseSelfAttention(d_model, num_heads)
        self.cross_attn = PositionWiseSelfAttention(d_model, num_heads)
        self.ff = PositionWiseFeedForward(d_model, d_big=d_ff)
        self.norm1 = nn.LayerNorm(d_model)
        self.norm2 = nn.LayerNorm(d_model)
        self.norm3 = nn.LayerNorm(d_model)
        self.droput = nn.Dropout(dropout)


    def forward(self, x, enc_out, source_mask, target_mask):
        self_attention = self.self_attn(x, x, x, target_mask)
        norm_1 = self.norm1(x + self.droput(self_attention))

        cross_attention = self.cross_attn(x, enc_out, enc_out, source_mask)

        norm_2 = (norm_1 + self.droput(cross_attention))
        feedforward = self.ff(norm_2)
        norm_3 = (norm_2 + self.droput(feedforward))

        return norm_3


In [25]:
#correct format to generate mask layers
# don't need a whole class - you are not maniputating or re-using the mask layers so you can chill
# part of transformer put together but you could probably write it by itself
import torch
import torch.nn as nn


'''class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()  # Ensure this is the first line in the __init__ method

        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([Decoder(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)
        


    def create_masks( self, source, target):
        #create boolean matrix and for every element, if it is 0 put false and non 0 put triue
        source_mask = (source != 0).unsqueeze(1).unsqueeze(2)
        target_mask = (source != 0).unsqueeze(1).unsqueeze(3)

        sequence_len =target.size(1)

        matrix_ones = torch.ones(1, sequence_len, sequence_len)
        upper_triangle_matrix = torch.triu(matrix_ones, diagonal = 1)
        #invert the matrix using some simple math - nice
        lower_triangle = 1- upper_triangle_matrix
        nopeak_mask = lower_triangle.bool()
        target_mask = nopeak_mask & target_mask

        return target_mask, source_mask
    
    def forward(self, x):

        source_mask, target_mask = self.create_masks(x, x)

        enc_bed = self.encoder_embedding(x)
        position_n_inbed = self.dropout(self.positional_encoding(enc_bed))

        encoding = position_n_inbed
        for layer in self.encoderlayers:
            encoding = layer(encoding, source_mask)

        dec_bed = self.decoder_embedding(x)

        dec_pos = self.dropout(self.positional_encoding(dec_bed))
        
        decoding = dec_pos
        for layer in self.decoder_layers:

            decoding = layer(decoding, encoding, source_mask, target_mask)

        output = self.final_linear(decoding)

        return output'''

class Transformer(nn.Module):
    def __init__(self, src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout):
        super(Transformer, self).__init__()  # Ensure this is the first line in the __init__ method

        self.encoder_embedding = nn.Embedding(src_vocab_size, d_model)
        self.decoder_embedding = nn.Embedding(tgt_vocab_size, d_model)
        self.positional_encoding = PositionEncoding(d_model, max_seq_length)

        self.encoder_layers = nn.ModuleList([EncoderLayer(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])
        self.decoder_layers = nn.ModuleList([Decoder(d_model, num_heads, d_ff, dropout) for _ in range(num_layers)])

        self.fc = nn.Linear(d_model, tgt_vocab_size)
        self.dropout = nn.Dropout(dropout)

    def create_masks(self, src, tgt):
        src_mask = (src != 0).unsqueeze(1).unsqueeze(2)
        tgt_mask = (tgt != 0).unsqueeze(1).unsqueeze(3)
        seq_length = tgt.size(1)
        nopeak_mask = (1 - torch.triu(torch.ones(1, seq_length, seq_length), diagonal=1)).bool()
        tgt_mask = tgt_mask & nopeak_mask
        return src_mask, tgt_mask


    def forward(self, source, target):
        source_mask, target_mask = self.create_masks(source, target)
        
        enc_bed = self.encoder_embedding(source)
        position_n_inbed = self.dropout(self.positional_encoding(enc_bed))

        encoding = position_n_inbed
        for i, layer in enumerate(self.encoder_layers):
            encoding = layer(encoding, source_mask)

        dec_bed = self.decoder_embedding(target)
        dec_pos = self.dropout(self.positional_encoding(dec_bed))

        decoding = dec_pos
        for i, layer in enumerate(self.decoder_layers):
            decoding = layer(decoding, encoding, source_mask, target_mask)

        output = self.fc(decoding)
        return output

''' def forward(self, source, target):
        source_mask, target_mask = self.create_masks(source, target)

        enc_bed = self.encoder_embedding(source)
        position_n_inbed = self.dropout(self.positional_encoding(enc_bed))

        encoding = position_n_inbed
        for layer in self.encoder_layers:
            encoding = layer(encoding, source_mask)

        dec_bed = self.decoder_embedding(target)
        dec_pos = self.dropout(self.positional_encoding(dec_bed))

        decoding = dec_pos
        for layer in self.decoder_layers:
            print(source_mask.size(), target_mask.size())
            decoding = layer(decoding, encoding, source_mask, target_mask)

        output = self.fc(decoding)
        return output'''



' def forward(self, source, target):\n        source_mask, target_mask = self.create_masks(source, target)\n\n        enc_bed = self.encoder_embedding(source)\n        position_n_inbed = self.dropout(self.positional_encoding(enc_bed))\n\n        encoding = position_n_inbed\n        for layer in self.encoder_layers:\n            encoding = layer(encoding, source_mask)\n\n        dec_bed = self.decoder_embedding(target)\n        dec_pos = self.dropout(self.positional_encoding(dec_bed))\n\n        decoding = dec_pos\n        for layer in self.decoder_layers:\n            print(source_mask.size(), target_mask.size())\n            decoding = layer(decoding, encoding, source_mask, target_mask)\n\n        output = self.fc(decoding)\n        return output'

In [26]:
import torch
import random

# Assuming you have a 'Transformer' class and necessary helper modules (like PositionalEncoding) defined elsewhere.

# Hyperparameters (these define the size and structure of your Transformer model)
src_vocab_size = 5000  # Size of the source vocabulary (e.g., number of unique characters/tokens in expressions)
tgt_vocab_size = 5000  # Size of the target vocabulary (e.g., number of unique characters/tokens in results)
d_model = 512         # Dimensionality of the embedding layer and the internal representations of the Transformer
num_heads = 8         # Number of attention heads in the multi-head attention mechanism
num_layers = 6        # Number of encoder (and potentially decoder) layers in the Transformer
d_ff = 2048         # Dimensionality of the feed-forward network within each Transformer layer
max_seq_length = 100  # Maximum length of the input and output sequences that the model can handle
dropout = 0.1         # Dropout probability for regularization

# Instantiate the Transformer model
# This creates an instance of your Transformer class with the specified hyperparameters.
# It initializes all the layers and parameters of the model.
transformer = Transformer(src_vocab_size, tgt_vocab_size, d_model, num_heads, num_layers, d_ff, max_seq_length, dropout)

# --- Process to Create Arithmetic Expression Data ---
batch_size = 64  # Number of arithmetic expressions to generate in this batch
max_len = max_seq_length  # Using the defined max sequence length for padding

def generate_arithmetic_expressions(num_samples, max_length):
    """
    Generates a list of simple arithmetic expressions and their results.

    Args:
        num_samples (int): The number of expression-result pairs to generate.
        max_length (int): The maximum length of the expression and result sequences (for padding).

    Returns:
        list: A list of tuples, where each tuple contains:
              - a list of tokens representing the arithmetic expression
              - a list of tokens representing the result
    """
    data = []
    operators = ["+", "-"]
    digits = "0123456789"
    for _ in range(num_samples):
        num1 = random.randint(1, 9)
        num2 = random.randint(1, 9)
        op = random.choice(operators)
        expression = f"{num1} {op} {num2}"
        try:
            result = str(eval(expression))
            src_sequence = list(expression)  # Split the expression into characters (tokens)
            tgt_sequence = list(result)      # Split the result into characters (tokens)

            # Pad sequences to max_length with a '<pad>' token (assuming you'll handle this in your vocabulary)
            src_sequence = src_sequence + ['<pad>'] * (max_length - len(src_sequence))
            tgt_sequence = tgt_sequence + ['<pad>'] * (max_length - len(tgt_sequence))

            data.append((src_sequence, tgt_sequence))
        except:
            continue  # Skip if there's an error in evaluation (shouldn't happen with simple +/-)
    return data

# Generate the arithmetic dataset
arithmetic_data = generate_arithmetic_expressions(batch_size, max_len)

# --- Process to Convert Text Data to Numerical Tensors ---
# You'll need to have a vocabulary (mapping from tokens to numbers) for both the source and target.
# Assuming you have 'src_vocab' and 'tgt_vocab' dictionaries created elsewhere.

def numericalize_sequence(sequence, vocab, max_length):
    """
    Converts a sequence of tokens into a list of numerical indices based on the vocabulary.
    Pads the sequence to the maximum length.

    Args:
        sequence (list): A list of tokens (e.g., characters).
        vocab (dict): A dictionary mapping tokens to their numerical indices.
        max_length (int): The maximum length of the sequence.

    Returns:
        torch.Tensor: A 1D tensor of numerical indices.
    """
    numericalized = [vocab.get(token, vocab.get('<unk>', 0)) for token in sequence] # Use <unk> if token not in vocab
    padded = numericalized + [vocab.get('<pad>', 0)] * (max_length - len(numericalized))
    return torch.tensor(padded)

# Assuming you have built your source and target vocabularies ('src_vocab' and 'tgt_vocab')
# based on the characters present in your arithmetic expressions and results.

# Example vocabulary creation (you might have a more sophisticated way of doing this)
all_chars = set()
for src, tgt in arithmetic_data:
    all_chars.update(src)
    all_chars.update(tgt)
all_chars.add('<pad>')
all_chars = sorted(list(all_chars))
char_to_index = {char: i for i, char in enumerate(all_chars)}
src_vocab = char_to_index
tgt_vocab = char_to_index # In this simple case, source and target vocab can be the same

# Convert the generated arithmetic data into numerical tensors
src_data = torch.stack([numericalize_sequence(item[0], src_vocab, max_len) for item in arithmetic_data])
tgt_data = torch.stack([numericalize_sequence(item[1], tgt_vocab, max_len) for item in arithmetic_data])

# Now 'src_data' and 'tgt_data' are PyTorch tensors containing the numerical representations
# of your arithmetic expressions and their corresponding results, ready to be used for training.



In [27]:
criterion = nn.CrossEntropyLoss(ignore_index=0)
optimizer = optim.Adam(transformer.parameters(), lr=0.0001, betas=(0.9, 0.98), eps=1e-9)

transformer.train()

for epoch in range(100):
    optimizer.zero_grad()
    output = transformer(src_data, tgt_data[:, :-1])
    loss = criterion(output.contiguous().view(-1, tgt_vocab_size), tgt_data[:, 1:].contiguous().view(-1))
    loss.backward()
    optimizer.step()
    print(f"Epoch: {epoch+1}, Loss: {loss.item()}")

KeyboardInterrupt: 

In [ ]:
import torch

# Ensure your Transformer model is in evaluation mode
transformer.eval()
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
transformer.to(device)

# Reuse the data generation function to create a validation set
def generate_arithmetic_expressions(num_samples=32, max_length=max_seq_length): # Smaller validation set
    data = []
    operators = ["+", "-"]
    digits = "0123456789"
    for _ in range(num_samples):
        num1 = random.randint(1, 9)
        num2 = random.randint(1, 9)
        op = random.choice(operators)
        expression = f"{num1} {op} {num2}"
        try:
            result = str(eval(expression))
            src_sequence = list(expression)
            tgt_sequence = list(result)
            src_sequence = src_sequence + ['<pad>'] * (max_length - len(src_sequence))
            tgt_sequence = tgt_sequence + ['<pad>'] * (max_length - len(tgt_sequence))
            data.append((src_sequence, tgt_sequence))
        except:
            continue
    return data

# Reuse the numericalization function
def numericalize_sequence(sequence, vocab, max_length):
    numericalized = [vocab.get(token, vocab.get('<unk>', 0)) for token in sequence]
    padded = numericalized + [vocab.get('<pad>', 0)] * (max_length - len(numericalized))
    return torch.tensor(padded).unsqueeze(0).to(device) # Add batch dimension

# Generate the validation dataset
validation_data = generate_arithmetic_expressions(num_samples=32)

# Evaluation loop
correct_predictions = 0
total_samples = len(validation_data)

with torch.no_grad(): # Disable gradient calculation during evaluation
    for src_text, true_tgt_text in validation_data:
        src_tensor = numericalize_sequence(src_text, src_vocab, max_seq_length)

        # **CORRECTION STARTS HERE**
        # Create a target tensor for inference (start with a <start> token)
        # Replace '<start>' with your actual start token if different
        tgt_tensor = torch.tensor([tgt_vocab.get('<start>', 0)]).unsqueeze(0).to(device)

        # Inference loop to generate the target sequence
        for _ in range(max_seq_length):  # Limit the generation length
            output = transformer(src_tensor, tgt_tensor)  # Pass both src and tgt
            # Get the last predicted token
            predicted_index = torch.argmax(output[:, -1, :], dim=-1)
            # Append the predicted token to the target sequence
            tgt_tensor = torch.cat([tgt_tensor, predicted_index.unsqueeze(0)], dim=1)
            # Stop if the predicted token is the <end> token
            # Replace '<end>' with your actual end token if different
            if predicted_index.item() == tgt_vocab.get('<end>', 0):
                break
        # **CORRECTION ENDS HERE**

        # Get the predicted sequence (argmax over the vocabulary dimension)
        predicted_indices = torch.argmax(output, dim=-1).squeeze(0).cpu().numpy()

        # Convert predicted indices back to tokens
        index_to_tgt_char = {i: char for char, i in tgt_vocab.items()}
        predicted_tokens = [index_to_tgt_char.get(idx, '<unk>') for idx in predicted_indices]

        # Stop prediction at the first padding token
        if '<pad>' in predicted_tokens:
            predicted_tokens = predicted_tokens[:predicted_tokens.index('<pad>')]

        # Stop true target at the first padding token
        if '<pad>' in true_tgt_text:
            true_tgt_text = true_tgt_text[:true_tgt_text.index('<pad>')]

        predicted_result = "".join(predicted_tokens)
        true_result = "".join(true_tgt_text)

        print(f"Input: {''.join(src_text).replace('<pad>', '')}, Predicted: {predicted_result}, True: {true_result}")

        if predicted_result == true_result:
            correct_predictions += 1

accuracy = correct_predictions / total_samples
print(f"\nValidation Accuracy: {accuracy * 100:.2f}%")

Input: 7 + 8, Predicted: , True: 15
Input: 2 + 6, Predicted: , True: 8
Input: 8 - 4, Predicted: , True: 4
Input: 4 - 7, Predicted: , True: -3
Input: 4 + 7, Predicted: , True: 11
Input: 2 + 1, Predicted: , True: 3


KeyboardInterrupt: 